In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-147342
Azure region: southcentralus
Subscription id: 976ee174-3882-4721-b90a-b5fef6b72f24
Resource group: aml-quickstarts-147342


In [2]:
# Create compute cluster
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

# Create a custom environment that encapsulates the training script's dependencies
# (1) Define your conda dependencies in a YAML file
# (2) Create an Azure ML environment from this Conda environment specification.

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- pip=20.2.4
- pip:
    - azureml-defaults
    - scikit-learn

Overwriting conda_dependencies.yml


In [4]:
from azureml.core import Environment

sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

In [5]:
from azureml.widgets import RunDetails
# from azureml.train.sklearn import SKLearn >> DEPRECATED. Use the ScriptRunConfig object with your own defined environment.
from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
    'C': uniform(0.001, 1.0), # For regularization
    'max_iter': choice(10, 50, 100) # Max number of epochs
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# DEPRECATED. Create a SKLearn estimator for use with train.py > Instead:
# Create a ScriptRunConfig object to specify the configuration details of your training job

est = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=cpu_cluster,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=est,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=20,
                                    max_concurrent_runs=4,
                                    max_duration_minutes=30)



In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hdr = exp.submit(config=hyperdrive_config)
RunDetails(hdr).show()
hdr.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_d5c00597-c3a2-4db7-a73f-9fe4332aaa44
Web View: https://ml.azure.com/runs/HD_d5c00597-c3a2-4db7-a73f-9fe4332aaa44?wsid=/subscriptions/976ee174-3882-4721-b90a-b5fef6b72f24/resourcegroups/aml-quickstarts-147342/workspaces/quick-starts-ws-147342&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-06-17T12:07:08.641475][API][INFO]Experiment created<END>\n""<START>[2021-06-17T12:07:09.183868][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-06-17T12:07:09.350577][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_d5c00597-c3a2-4db7-a73f-9fe4332aaa44
Web View: https://ml.azure.com/runs/HD_d5c00597-c3a2-4db7-a73f-9fe4332aaa44?wsid=/subscriptions/976ee174-3882-4721-b90a-b5fef6b72f24/resourcegroups/aml-quickstarts-147342/workspaces/quick-starts-ws-147342&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Warnings:
{
  "error

{'runId': 'HD_d5c00597-c3a2-4db7-a73f-9fe4332aaa44',
 'target': 'cpucluster',
 'status': 'Canceled',
 'startTimeUtc': '2021-06-17T12:07:08.315696Z',
 'endTimeUtc': '2021-06-17T12:15:08.834331Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": null,\n    "additionalInfo": null\n  },\n  "correlation": null,\n  "environment": null,\n  "location": null,\n  "time": "0001-01-01T00:00:00+00:00",\n  "componentName": null\n}'}],
 'properties': {'primary_m

In [7]:
# Get your best run and save the model from that run.
best_run = hdr.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print('\n learning rate:', parameter_values[3])
print('\n keep probability:', parameter_values[5])
print('\n batch size:', parameter_values[7])

from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

model = run.register_model(model_name='sklearn-logistic-regression', 
                           model_path='outputs/model.joblib')


AttributeError: 'NoneType' object has no attribute 'get_metrics'

In [ ]:
hdr.get_metrics()

In [ ]:
[att for att in dir(hdr) if 'metr' in att.lower()]

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

In [ ]:
# Delete() is used to deprovision and delete the AmlCompute target. 

cpu_cluster.delete()